### Extract all `.tar` files in the current directory
Click to expand code.

In [1]:
# <details>
# <summary>Show code</summary>
# ...existing code...
# </details>

In [2]:
# <details>
# <summary>Show code</summary>
import tarfile
import os
from glob import glob

# Find all .tar files in the current directory
tar_files = glob("*.tar")

for tar_filename in tar_files:
    with tarfile.open(tar_filename, "r") as tar:
        tar.extractall(path=os.getcwd())
    print(f"Extracted {tar_filename} in {os.getcwd()}")
# </details>

/var/folders/d9/rdtj7fbj1lz5ybbbf6vstzbm0000gn/T/ipykernel_65671/2475519837.py:12: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=os.getcwd())


Extracted 2025-01-17.tar in /Users/code/source/aia_exam


### Connect to SQLite database and show sample data
Click to expand code.

In [3]:
# <details>
# <summary>Show code</summary>
import sqlite3
import pandas as pd
import os

# Path to the SQLite database
db_path = os.path.join("db", "aia_big.db")

# Connect to the database
conn = sqlite3.connect(db_path)

# Get table names
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in database:", tables['name'].tolist())

# For demonstration, use the first table found
table_name = tables['name'][0]

# Get column names
columns = pd.read_sql_query(f"PRAGMA table_info({table_name});", conn)
print("Column names:", columns['name'].tolist())

# Get a sample of 10 rows
sample = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 10;", conn)
print("Sample rows:")
print(sample)

conn.close()
# </details>

Tables in database: ['gather']
Column names: ['date', 'instrument', 'buy', 'sell']
Sample rows:
                         date instrument         buy        sell
0  2025-01-10 21:59:05.151250    EUR_USD    1.024856    1.023995
1  2025-01-10 21:59:05.133156    EUR_GBP    0.839559    0.838867
2  2025-01-10 21:59:05.125782    EUR_AUD    1.668097    1.666619
3  2025-01-10 21:59:05.127208    EUR_CAD    1.477946    1.476747
4  2025-01-10 21:59:05.147446    EUR_SGD    1.405943    1.404639
5  2025-01-10 21:59:05.128255    EUR_CHF    0.939286    0.938533
6  2025-01-10 21:59:05.136770    EUR_JPY  161.686143  161.549429
7  2025-01-10 21:59:05.300552    EUR_NZD    1.845037    1.843097
8  2025-01-10 21:59:05.134283    EUR_HKD    7.982760    7.967652
9  2025-01-10 21:59:05.130820    EUR_CZK   25.099497   25.047804


### Query USD_CAD rows and shift date by -4 hours
Click to expand code.

In [4]:
# <details>
# <summary>Show code</summary>
import sqlite3
import pandas as pd
import os

# Path to the SQLite database
db_path = os.path.join("db", "aia_big.db")

# Connect to the database
conn = sqlite3.connect(db_path)

# Query for all rows where instrument is USD_CAD, ordered by date, and shift date by -4 hours
query = f"""
SELECT 
    datetime(date, '-4 hours') as date_shifted,
    instrument,
    buy,
    sell
FROM {table_name}
WHERE instrument = 'USD_CAD'
ORDER BY date;
"""
usdcad_shifted = pd.read_sql_query(query, conn)
print("Rows where instrument is USD_CAD, date shifted -4 hours, ordered by date:")
print(usdcad_shifted)

conn.close()
# </details>

Rows where instrument is USD_CAD, date shifted -4 hours, ordered by date:
               date_shifted instrument       buy      sell
0       2025-01-10 17:59:05    USD_CAD  1.442673  1.441825
1       2025-01-12 18:04:55    USD_CAD  1.442913  1.441915
2       2025-01-12 18:05:00    USD_CAD  1.442913  1.441965
3       2025-01-12 18:05:00    USD_CAD  1.442913  1.441825
4       2025-01-12 18:05:01    USD_CAD  1.442913  1.441835
...                     ...        ...       ...       ...
390817  2025-01-17 17:58:54    USD_CAD  1.448583  1.447805
390818  2025-01-17 17:58:54    USD_CAD  1.448563  1.447795
390819  2025-01-17 17:58:54    USD_CAD  1.448583  1.447795
390820  2025-01-17 17:58:54    USD_CAD  1.448583  1.447775
390821  2025-01-17 17:59:00    USD_CAD  1.448583  1.447525

[390822 rows x 4 columns]


### Resample, compute EMA/TEMA, detect crosses, and plot
Click to expand code.

In [6]:
# <details>
# <summary>Show code</summary>
import pandas as pd
from pyecharts.charts import Line
from pyecharts import options as opts
from pyecharts.globals import CurrentConfig, NotebookType
from datetime import timedelta
from pyecharts.render import make_snapshot
from snapshot_selenium import snapshot

import ipywidgets as widgets
from IPython.display import display, clear_output

# Ensure date_shifted is datetime and sorted
usdcad_shifted['date_shifted'] = pd.to_datetime(usdcad_shifted['date_shifted'])
usdcad_shifted = usdcad_shifted.sort_values('date_shifted')

# Set the time window for EMA/EMA3 in minutes
window_minutes = 15

# Resample to 1-minute intervals, using the last value in each minute
df_min = usdcad_shifted.set_index('date_shifted').resample('1T').last().dropna(subset=['buy'])

# Calculate the span for EMA to approximate a 15-minute window
# alpha = 2 / (span + 1) => span = (2 / alpha) - 1
# For a 15-minute EMA on 1-minute data, span = 15
df_min['xee'] = df_min['buy'].ewm(span=window_minutes, adjust=False).mean()

# Triple EMA (TEMA) calculation for 15 minutes
ema1 = df_min['buy'].ewm(span=window_minutes, adjust=False).mean()
ema2 = ema1.ewm(span=window_minutes, adjust=False).mean()
ema3 = ema2.ewm(span=window_minutes, adjust=False).mean()
df_min['xer'] = 3 * (ema1 - ema2) + ema3

# Prepare data for plotting (downsample for performance if needed)
plot_df = df_min.iloc[::max(1, len(df_min)//2000)]  # limit to ~2000 points

# Detect cross up and cross down events
cross_up = (plot_df['xer'] > plot_df['xee']) & (plot_df['xer'].shift(1) <= plot_df['xee'].shift(1))
cross_dn = (plot_df['xer'] < plot_df['xee']) & (plot_df['xer'].shift(1) >= plot_df['xee'].shift(1))

plot_df['cross_up'] = plot_df['buy'].where(cross_up)
plot_df['cross_dn'] = plot_df['buy'].where(cross_dn)

# Set notebook type for pyecharts
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

# Calculate min/max for y-axis to avoid starting at 0
ymin = min(plot_df[['buy', 'xee', 'xer']].min()) * 0.999
ymax = max(plot_df[['buy', 'xee', 'xer']].max()) * 1.001

# Create ECharts line chart with custom colors, solid lines, and white background (remove scale argument)
line = (
    Line(init_opts=opts.InitOpts(bg_color="#ffffff"))
    .add_xaxis(plot_df.index.strftime('%Y-%m-%d %H:%M:%S').tolist())
    .add_yaxis(
        "Buy",
        plot_df['buy'].tolist(),
        is_smooth=True,
        linestyle_opts=opts.LineStyleOpts(width=1, color="#888888", type_="solid"),
        label_opts=opts.LabelOpts(is_show=False),
        symbol="none"
    )
    .add_yaxis(
        "EMA 15min (xee)",
        plot_df['xee'].tolist(),
        is_smooth=True,
        linestyle_opts=opts.LineStyleOpts(width=2, color="#1976d2", type_="solid"),
        label_opts=opts.LabelOpts(is_show=False),
        symbol="none"
    )
    .add_yaxis(
        "Triple EMA 15min (xer)",
        plot_df['xer'].tolist(),
        is_smooth=True,
        linestyle_opts=opts.LineStyleOpts(width=2, color="#8e24aa", type_="solid"),
        label_opts=opts.LabelOpts(is_show=False),
        symbol="none"
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="USD_CAD Buy Price with EMA and Triple EMA (15min)"),
        xaxis_opts=opts.AxisOpts(type_="category", name="Date", axislabel_opts=opts.LabelOpts(rotate=45)),
        yaxis_opts=opts.AxisOpts(name="Buy Price", min_="dataMin", max_="dataMax"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        datazoom_opts=[opts.DataZoomOpts()],
        legend_opts=opts.LegendOpts(is_show=False)
    )
)

line.render_notebook()
# </details>

/var/folders/d9/rdtj7fbj1lz5ybbbf6vstzbm0000gn/T/ipykernel_65671/2173586513.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_min = usdcad_shifted.set_index('date_shifted').resample('1T').last().dropna(subset=['buy'])
/var/folders/d9/rdtj7fbj1lz5ybbbf6vstzbm0000gn/T/ipykernel_65671/2173586513.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plot_df['cross_up'] = plot_df['buy'].where(cross_up)
/var/folders/d9/rdtj7fbj1lz5ybbbf6vstzbm0000gn/T/ipykernel_65671/2173586513.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or